In [ ]:
import geopandas as gpd
import pandas as pd
from h3 import h3
from sqlalchemy import create_engine
import numpy as np
import psycopg2

In [ ]:
def crear_filtro_trx_seguidas(dt, minutos = 3, fecha_col = 'FECHATRX'):
    '''
    esta funcion toma el dataset de viajes, agrupa por tarjeta,
    ordena por fecha y elimina las transacciones 'simultaneas', 
    es decir, menos de 3 minutos o el parametro que se corra
    '''
    dt = dt.sort_values(by=[fecha_col])
    dt['delta'] = (dt[fecha_col].shift(-1) - dt[fecha_col])#.fillna(pd.Timedelta(10**12))
    dt['trx_no_seguidas'] = dt['delta'] /np.timedelta64(1, 'm') > minutos
    #la ultima transaccion va a tener un delta de NaT, va True
    dt['trx_no_seguidas'].iloc[-1] = True
    dt = dt.loc[dt.trx_no_seguidas,:]
    return dt

def crear_tramo_id(dt):
    dt['tramo_id'] = range(len(dt))
    return dt

In [ ]:
DB_USERNAME = 'sube_user'
DB_PASSWORD = 'sube_pass'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'sube'
DB_SCHEMA = 'public'

In [ ]:
resolucion_h3 = 11

In [ ]:
# Conectar a la db
conn = psycopg2.connect(user = DB_USERNAME,
                                      password = DB_PASSWORD,
                                      host = DB_HOST,
                                      port = DB_PORT,
                                      database = DB_NAME)

In [ ]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'
    .format(DB_USERNAME, DB_PASSWORD, DB_HOST,
            DB_PORT, DB_NAME))

In [ ]:
# traer trx de tarjetas que hayan pasado por la B
q = """
select t."ID_TARJETA",t."LATITUDE",t."LONGITUDE",t."FECHATRX",t."LINEA",t."h3_res_%i"
from trx t
where t."ID_TARJETA" in (
    select distinct t."ID_TARJETA" 
    from trx t
    where t."LINEA" = 'LINEA B'

);
"""%resolucion_h3
trx = pd.read_sql(q, conn)
trx.shape

In [ ]:
trx[trx.ID_TARJETA==955653]

In [ ]:
# elimino tarjetas con una unica transaccion 
tarjetas_unica_trx = (trx.groupby('ID_TARJETA').count()['LINEA'] == 1)
tarjetas_unica_trx = tarjetas_unica_trx.index[tarjetas_unica_trx]
print('eliminadas por viaje unico %.2f por ciento'%(len(tarjetas_unica_trx)/len(trx.ID_TARJETA.unique())*100))
trx = trx.loc[~trx.ID_TARJETA.isin(tarjetas_unica_trx),:]
trx.head()

In [ ]:
trx[trx.ID_TARJETA==955653]

In [ ]:
#eliminar transacciones seguidas
print(trx.shape)
trx = trx.groupby('ID_TARJETA').apply(crear_filtro_trx_seguidas).copy()
trx = trx.reset_index(drop=True)
print(trx.shape)

In [ ]:
trx[trx.ID_TARJETA==955653]

In [ ]:
trx = trx.drop('trx_no_seguidas',axis=1)

In [ ]:
trx[trx.ID_TARJETA==955653]

In [ ]:
trx = trx.rename(columns = {'ID_TARJETA':'tarjeta',
                    'LINEA':'linea',
                    'LATITUDE':'lat_o',
                    'LONGITUDE':'lon_o',
                    'FECHATRX':'fecha',
                    'h3_res_%i'%resolucion_h3:'h3_o'
                    })

In [ ]:
trx.head()

In [ ]:
trx[trx.tarjeta==955653]

In [ ]:
trx = trx.groupby('tarjeta').apply(crear_tramo_id)


In [ ]:
trx[trx.tarjeta==955653]

In [ ]:
conn.close()

In [ ]:
trx.to_csv('../data/trx_linea_b.csv',index=False)

In [ ]:
trx.to_sql('trx_linea_b', engine, schema=DB_SCHEMA,index=False,method='multi')

In [ ]:
# traer trx de tarjetas que hayan pasado por la B
q = """
select *
from trx_linea_b tlb;
"""
trx = pd.read_sql(q, conn)
trx.shape

In [ ]:
trx['delta'] = pd.to_timedelta(trx['delta'])
trx.head()

In [ ]:
tabla_tramos = trx.reindex(columns = ['tarjeta','tramo_id','linea','fecha','lat_o','lon_o','delta','h3_o'])
tabla_tramos['lat_d'] = None
tabla_tramos['lon_d'] = None
tabla_tramos['h3_d'] = None
tabla_tramos['viaje_id'] = None
tabla_tramos.head(20)

In [ ]:
tabla_tramos.to_sql('tramos_linea_b', engine, schema=DB_SCHEMA,index=False,method='multi')

In [ ]:
# para volver el valor a time delta desde la db usar pd.to_timedelta(delta)